Make rivers constant and rivers monthly file
Temperature in not set in these files
They have flux, area, depth 

In [1]:
import netCDF4 as NC
import numpy as np
import matplotlib.pyplot as plt

from salishsea_tools import rivertools, river_201702, river_downbyone2, nc_tools

%matplotlib inline

In [2]:
import importlib
importlib.reload(river_downbyone2)

<module 'salishsea_tools.river_downbyone2' from '/ocean/sallen/allen/research/Meopar/Tools/SalishSeaTools/salishsea_tools/river_downbyone2.py'>

In [3]:
#what type of file would you like to create?
#e.g. 'constant' - yearly average used for all months
#     'monthly' - monthly averages used for each month
rivertype = 'monthly'

In [4]:
# what coordinates are you using?
coords = 'coordinates_seagrid_SalishSea.nc'
coords_file = '../../../NEMO-forcing/grid/'+coords
# where is the river information?
prop_dict_name ='river_downbyone2'
prop_dict = river_downbyone2.prop_dict

In [5]:
# what file name do you want to produce?
netcdf_file = 'rivers_month_extended.nc'

In [6]:
#get dimensions for netcdf files
fB = NC.Dataset(coords_file, 'r')
lat = fB.variables['nav_lat']
lon = fB.variables['nav_lon']
e1t = fB.variables['e1t'][0,:]
e2t = fB.variables['e2t'][0,:]
area = e1t*e2t
fB.close()

In [7]:
#get watershed fluxes from netcdf file
if rivertype == 'constant':
    fluxfile = NC.Dataset('../../../nemo-forcing/rivers/Salish_allrivers_cnst.nc','r')
    #inialise the runoff and run_depth arrays
    runoff = np.zeros_like(area)
if rivertype == 'monthly':
    fluxfile = NC.Dataset('../../../nemo-forcing/rivers/Salish_allrivers_monthly.nc','r')
    #inialise the runoff and run_depth arrays
    runoff = np.zeros((12, area.shape[0], area.shape[1]))

In [8]:
#list of watersheds we are including
names = ['skagit', 'fraser', 'evi_n', 'howe', 'bute', 'puget', 'jdf', 'evi_s', 'jervis', 'toba']

In [9]:
run_depth = np.ones_like(runoff)
run_temp = np.empty_like(runoff)
for name in range(0,len(names)):
    watershedname = names[name]
    print (watershedname)
    Flux = fluxfile.variables[watershedname][:]
    if rivertype == 'constant':
        Flux = float(Flux)
    runoff_orig = np.copy(runoff)
    runoff, run_depth, run_temp = rivertools.put_watershed_into_runoff(rivertype, area,
                                            Flux, runoff, run_depth, run_temp,
                                            prop_dict[watershedname])
    if rivertype == 'constant':
        rivertools.check_sum(runoff_orig, runoff, Flux, area)
    if rivertype == 'monthly':
        rivertools.check_sum_monthly(runoff_orig, runoff, Flux, area)

skagit
937.969310164 937.969319661 0.999999989875
fraser
3539.76580088 3539.81380208 0.999986439626
evi_n
249.918898314 638.888590495 0.391177588757
howe
571.427951034 571.457763672 0.999947830549
bute
547.815330845 547.855875651 0.999925993664
puget
503.333033348 503.333048503 0.999999969892
jdf
411.162093321 411.162068685 1.00000005992
evi_s
330.77776717 330.786621094 0.999973233731
jervis
296.9668444 296.966837565 1.00000002301
toba
269.874062982 269.932454427 0.999783681272


In [10]:
# set up titles
netcdf_title = rivertype+' rivers for '+prop_dict_name
netcdf_comment = netcdf_title+' on '+coords
notebook = 'MakeClimatologyRiverFilesWithArea.ipynb'

nemo = NC.Dataset(netcdf_file, 'w')
nc_tools.init_dataset_attrs(nemo, netcdf_title, notebook,
                           'NEMO-forcing/rivers/'+netcdf_file, netcdf_comment)

# dimensions
nemo.createDimension('x', area.shape[1])
nemo.createDimension('y', area.shape[0])
nemo.createDimension('time_counter', None)
    
# variables
# latitude and longitude
nav_lat = nemo.createVariable('nav_lat', 'float32', ('y', 'x'), zlib=True)
nav_lat.setncattr('units', 'degrees_north')
nav_lat.setncattr('long_name', 'latitude')
nav_lat = lat
nav_lon = nemo.createVariable('nav_lon','float32',('y','x'),zlib=True)
nav_lon.setncattr('units', 'degrees_east')
nav_lon.setncattr('long_name', 'longitude')
nav_lon = lon
# area
nav_area = nemo.createVariable('area', 'float32', ('y', 'x'), zlib=True)
nav_area.setncattr('units', 'm2')
nav_area.setncattr('long_name', 'horizontal_grid_cell_area')
nav_area = area

# time
time_counter = nemo.createVariable('time_counter', 'float32', ('time_counter'),zlib=True)
time_counter.setncattr('units', 'month')
time_counter.setncattr('long_name', 'time')
if rivertype == 'monthly':
    time_counter[0:12] = range(1,13)
elif rivertype == 'constant':
    time_counter[0] = 1
else:
    print('problem!!!')

# runoff
rorunoff = nemo.createVariable('rorunoff', 'float32', ('time_counter','y','x'), zlib=True)
rorunoff.setncattr('units', 'kg m-2 s-1')
rorunoff.setncattr('long_name', 'runoff_flux')
if rivertype == 'monthly':
    rorunoff[0:12,:] = runoff
else:
    rorunoff[0, :] = runoff

# depth
rodepth = nemo.createVariable('rodepth','float32',('y','x'),zlib=True)
rodepth.setncattr('units', 'm')
rodepth.setncattr('long_name', 'maximum_runoff_depth')
if rivertype == 'monthly':
    rodepth[:] = run_depth[0,:,:]
else:
    rodepth[:] = run_depth
    
nc_tools.show_dataset_attrs(nemo)
nemo.close()

file format: NETCDF4
Conventions: CF-1.6
title: monthly rivers for river_downbyone2
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/salishsea/tools/src/tip/MakeClimatologyRiverFilesWithArea.ipynb
references: REQUIRED
history: [2017-05-13 15:41:10] Created netCDF4 zlib=True dataset.
comment: monthly rivers for river_downbyone2 on coordinates_seagrid_SalishSea.nc
file format: NETCDF4
Conventions: CF-1.6
title: monthly rivers for river_downbyone2
institution: Dept of Earth, Ocean & Atmospheric Sciences, University of British Columbia
source: https://bitbucket.org/salishsea/tools/src/tip/MakeClimatologyRiverFilesWithArea.ipynb
references: REQUIRED
history: [2017-05-13 15:41:10] Created netCDF4 zlib=True dataset.
comment: monthly rivers for river_downbyone2 on coordinates_seagrid_SalishSea.nc


**Your file has now been created.  Move it into NEMO-forcing/Rivers and commit**